In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import pysewer
import matplotlib.pyplot as plt
import networkx as nx
from pysewer.config.settings import load_config, config_to_dataframe, view_default_settings  # load the the settings file 
import geopandas as gpd
from shapely.geometry import base
from shapely import wkt

load_dotenv()
WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')

# get the currene working directory
CWD = Path.cwd()
if CWD != WORKSPACE_DIR:
    os.chdir(WORKSPACE_DIR)
    print(f'Changed working directory to {WORKSPACE_DIR}')


Changed working directory to /Users/despot/Nextcloud/Cloud/Python_Projects/pysewer_dev


In [3]:
# load the gpkg file with the blocks to generate the sewer network
blocks = gpd.read_file("./data/combined_sewer/processed_blocks_industry.gpkg")

blocks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   block_id                   18 non-null     object  
 1   pop_density                18 non-null     object  
 2   area                       18 non-null     object  
 3   population                 18 non-null     object  
 4   population_density         18 non-null     object  
 5   sealed_area                18 non-null     object  
 6   unsealed_area              18 non-null     object  
 7   largest_potential_area     18 non-null     object  
 8   sealing_degree             18 non-null     object  
 9   largest_unsealed_geometry  18 non-null     object  
 10  mean_slope                 18 non-null     object  
 11  max_diameter               18 non-null     object  
 12  width_rectangle            18 non-null     object  
 13  width_square               18

In [4]:
blocks.head()

,block_id,pop_density,area,population,population_density,sealed_area,unsealed_area,largest_potential_area,sealing_degree,largest_unsealed_geometry,mean_slope,max_diameter,width_rectangle,width_square,form_factor,geometry
0,1,0.00015503640419670513,8066.800409887563,1.2506477289214748,0.00015503640419670513,3218.122172478661,4973.461539326657,4388.348417062244,0.39893415095955204,POLYGON ((479966.2433594299363904 5249138.6669...,7.690000057220459,135.09349544657982,59.71272253501966,89.81536845043594,0.5627850504871243,"POLYGON ((479937.639 5249115.735, 479959.542 5..."
1,2,0.0001559274206901419,10518.315250913614,1.6400937670807425,0.0001559274206901419,6875.079186683008,3730.0961544973584,3218.1221725030546,0.6536293144556437,POLYGON ((480000.4519291829201393 5249036.0412...,1.4800000190734863,153.57611952519272,68.48926306663311,102.55883799514118,0.5678176945465917,"POLYGON ((480041.939 5249080.679, 480140.738 5..."
2,3,0.00015998255498338453,22266.31438593335,3.5622218655249083,0.00015998255498338453,17407.115387598755,4754.044118461572,1682.2002265295998,0.7817690474448535,POLYGON ((480094.5254960035090335 5248941.9677...,2.7200000286102295,213.84980099803488,104.12127709269161,149.21901482697623,0.6199272892657994,"POLYGON ((480142.243 5248994.905, 480142.307 5..."
3,4,0.0001633716897378045,2218.4160711845557,0.3624263820909224,0.0001633716897378045,1316.5045250989106,950.8088237015735,438.83484170826534,0.5934434672554205,POLYGON ((480231.3597750153276138 5248890.6548...,2.180000066757202,70.61737958732432,31.414590631209954,47.100064449898106,0.566408712799404,"POLYGON ((480229.014 5248888.878, 480229.026 5..."
4,5,0.00018087581351951317,1969.7453178026244,0.3562792867838017,0.0001808758135195132,1901.6176473787546,73.13914027807355,73.13914027807355,0.9654129547570797,POLYGON ((480222.8076325770816766 5248822.2377...,7.730000019073486,68.14956077294806,28.90327238300432,44.38181291703421,0.540000389626882,"POLYGON ((480259.164 5248863.233, 480287.288 5..."


In [5]:
# create a function to convert to the columns to the correct data type (shapely objects and int, floats)
import geopandas as gpd
from shapely import wkt
import json
import pandas as pd

def load_and_restore_blocks_gdf(filepath, metadata_path, file_format="GeoJSON"):
    """
    Load a GeoDataFrame and restore its original data types based on stored metadata.

    Parameters:
    - filepath: Path to the saved GeoDataFrame.
    - metadata_path: Path to the metadata file (JSON).
    - file_format: Format of the saved GeoDataFrame ('GeoJSON' or 'GPKG').

    Returns:
    - GeoDataFrame with restored original data types.
    """
    # Load the GeoDataFrame
    if file_format == "GeoJSON":
        df = gpd.read_file(filepath)
    elif file_format == "GPKG":
        df = gpd.read_file(filepath, driver="GPKG")
    else:
        raise ValueError("Unsupported file format. Please use 'GPKG' or 'GeoJSON'.")

    # Load the metadata
    with open(metadata_path, 'r') as f:
        dtype_metadata = json.load(f)

    # Restore original data types
    for col, dtype in dtype_metadata.items():
        # print(dtype)
        if col in df.columns:
            if dtype == 'geometry':
                print(col)
            # Convert WKT strings back to geometries
                # Handle cases where the data might be either a WKT string or already a geometry
                df[col] = df[col].apply(lambda x: wkt.loads(x) if isinstance(x, str) else x)
                df[col] = gpd.GeoSeries(df[col])
            elif dtype == 'int64':
                df[col] = pd.to_numeric(df[col], downcast='integer')
            elif dtype == 'float64':
                df[col] = pd.to_numeric(df[col], downcast='float')
            else:
                df[col] = df[col].astype(dtype)
        else:
            print(f"Warning: Column '{col}' not found in the loaded DataFrame.")

    return df

# Example usage:
df_restored = load_and_restore_blocks_gdf("./data/combined_sewer/processed_blocks_industry.gpkg", "./data/combined_sewer/processed_blocks_industry_meta_data.json", file_format="GPKG")

geometry
largest_unsealed_geometry


In [6]:
df_restored.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   block_id                   18 non-null     int8    
 1   pop_density                18 non-null     float32 
 2   area                       18 non-null     float64 
 3   population                 18 non-null     float32 
 4   population_density         18 non-null     float32 
 5   sealed_area                18 non-null     float32 
 6   unsealed_area              18 non-null     float32 
 7   largest_potential_area     18 non-null     float32 
 8   sealing_degree             18 non-null     float32 
 9   largest_unsealed_geometry  18 non-null     geometry
 10  mean_slope                 18 non-null     float32 
 11  max_diameter               18 non-null     float32 
 12  width_rectangle            18 non-null     float32 
 13  width_square               18

In [8]:
# calculate the overall area of the blocks
total_area = blocks['geometry'].area.sum()      # in square meters

# convert to hectares
total_area = total_area / 10000
print(f'The total area of the blocks is {total_area} hectares')


The total area of the blocks is 16.903750818742434 hectares
